In [ ]:
!git clone https://github.com/dipandhali2021/Enhancer_BiLSTMAtt-ResNet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#choose from 'atomic' | 'EIIP' | 'BFDNA' | 'numeric'
DNA_encoding_scheme='BFDNA'

In [ ]:
with  open("/content/Enhancer_BiLSTMAtt-ResNet/data/human_mouse/enhancer.train.txt") as f:
       enhancer_train = f.readlines()
       enhancer_train = [s.strip() for s in enhancer_train]
with  open("/content/Enhancer_BiLSTMAtt-ResNet/data/human_mouse/enhancer.test.txt") as f:
       enhancer_test = f.readlines()
       enhancer_test = [s.strip() for s in enhancer_test]
with  open("/content/Enhancer_BiLSTMAtt-ResNet/data/human_mouse/non.enhancer.train.txt") as f:
       non_enhancer_train = f.readlines()
       non_enhancer_train = [s.strip() for s in non_enhancer_train]
with  open("/content/Enhancer_BiLSTMAtt-ResNet/data/human_mouse/non.enhancer.test.txt") as f:
       non_enhancer_test = f.readlines()
       non_enhancer_test = [s.strip() for s in non_enhancer_test]


In [ ]:
def remove_header(data):
    data_new = []
    for i in range(1,len(data),2):
        data_new.append(data[i])
    
    return data_new

In [ ]:
def ensure_fixed_length(sequences, length=200):
    fixed_length_sequences = []
    for seq in sequences:
        # Strip to length of 200
        if len(seq) > length:
            fixed_length_sequences.append(seq[:length])
        else:
            # If shorter, pad with 'N' (or other character of your choice)
            fixed_length_sequences.append(seq.ljust(length, 'N'))  # Use 'N' for padding
    return fixed_length_sequences

In [ ]:
enhancer_train = remove_header(enhancer_train)
non_enhancer_train = remove_header(non_enhancer_train)
enhancer_test = remove_header(enhancer_test)
non_enhancer_test = remove_header(non_enhancer_test)



enhancer_train = ensure_fixed_length(enhancer_train, length=200)
non_enhancer_train = ensure_fixed_length(non_enhancer_train, length=200)
enhancer_test = ensure_fixed_length(enhancer_test, length=200)
non_enhancer_test = ensure_fixed_length(non_enhancer_test, length=200)


print(len(enhancer_train),len(enhancer_train[0]))
print(len(enhancer_test),len(enhancer_test[0]))
print(len(non_enhancer_train),len(non_enhancer_train[0]))
print(len(non_enhancer_test),len(non_enhancer_test[0]))
train_x = np.concatenate([enhancer_train, non_enhancer_train], axis=0)
test_x = np.concatenate([enhancer_test, non_enhancer_test], axis=0)
print(len(train_x),len(test_x))


In [ ]:
train_y = np.concatenate([np.ones((len(enhancer_train),)), np.zeros((len(non_enhancer_train),))], axis=0)  
test_y = np.concatenate([np.ones((len(enhancer_test),)), np.zeros((len(non_enhancer_test),))], axis=0)
print(train_y.shape,test_y.shape)

In [ ]:
def encode_matrix(seq_matrix, encoding_scheme):
    """Encodes DNA sequences using the specified encoding scheme."""
    if encoding_scheme == 'BFDNA':
        # BFDNA Encoding
        def compute_bfdna(sequence):
            total_length = len(sequence)
            base_counts = {base: sequence.count(base) for base in 'ACGTN'}
            return [base_counts[base] / total_length for base in sequence]

        def scale_and_round(sequence_bfdna, scale_factor=100):
            return [round(value * scale_factor) for value in sequence_bfdna]
        
        return [scale_and_round(compute_bfdna(sequence)) for sequence in seq_matrix]

    elif encoding_scheme == 'numeric':
        # Numeric Mapping
        ind_to_char = ['A', 'T', 'C', 'G', 'N']
        char_to_ind = {char: i for i, char in enumerate(ind_to_char)}
        return [[char_to_ind[i] for i in s] for s in seq_matrix]

    elif encoding_scheme == 'atomic':
        # Atomic Mapping
        char_to_atomic_number = {'A': 70, 'T': 58, 'C': 78, 'G': 66, 'N': 0}
        return [[char_to_atomic_number[i] for i in s] for s in seq_matrix]

    elif encoding_scheme == 'EIIP':
        # EIIP Mapping
        char_to_eiip = {'A': 0.1260, 'C': 0.1340, 'G': 0.0806, 'T': 0.1335, 'N': 0.0}
        
        def scale_eiip(sequence_eiip, scale_factor=100):
            return [round(value * scale_factor, 2) for value in sequence_eiip]
        
        return [scale_eiip([char_to_eiip[i] for i in s]) for s in seq_matrix]

    else:
        raise ValueError(f"Unknown encoding scheme: {encoding_scheme}")


In [ ]:
train_x = encode_matrix(train_x,encoding_scheme=DNA_encoding_scheme)
test_x = encode_matrix(test_x,encoding_scheme=DNA_encoding_scheme)
train_x = np.array(train_x)
test_x = np.array(test_x)

In [ ]:
def sn_sp_acc_mcc(true_label, predict_label, pos_label=1):
    import numpy as np
    import math

    pos_num = np.sum(true_label == pos_label)
    neg_num = true_label.shape[0] - pos_num

    tp = np.sum((true_label == pos_label) & (predict_label == pos_label))
    tn = np.sum((true_label != pos_label) & (predict_label != pos_label))
    fn = pos_num - tp
    fp = neg_num - tn

    # Sensitivity (Recall) and Specificity
    sn = tp / pos_num if pos_num > 0 else 0
    sp = tn / neg_num if neg_num > 0 else 0

    # Accuracy
    acc = (tp + tn) / (pos_num + neg_num) if (pos_num + neg_num) > 0 else 0

    # Precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0

    # CSI
    csi = precision + sn - 1

    # G-mean
    gmean = math.sqrt(sn * sp)

    # MCC
    tp = np.array(tp, dtype=np.float64)
    tn = np.array(tn, dtype=np.float64)
    fp = np.array(fp, dtype=np.float64)
    fn = np.array(fn, dtype=np.float64)

    mcc_denominator = ((tp + fp) * (fp + tn) + (tp + fn) * (fn + tn))
    mcc = (2 * (tp * tn - fp * fn)) /  np.sqrt(mcc_denominator) if mcc_denominator > 0 else 0

    # Kappa
    kappa = (2 * (tp * tn - fp * fn)) / mcc_denominator if mcc_denominator > 0 else 0

    return sn, sp, acc, mcc, kappa, csi, gmean


In [ ]:
def define_model():
    maxlen = 200
    max_features = 100 
    embedding_dims = 128
    input = Input(shape=(maxlen,))
    embedding = Embedding(max_features, embedding_dims, input_length=maxlen)(input)

    # Convolutional branch
    conv = Conv1D(64, kernel_size=5, activation='relu')(embedding)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = GlobalMaxPool1D()(conv)

    # LSTM branch
    lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding)
    lstm = GlobalMaxPool1D()(lstm)

    # Combine branches
    combined = Concatenate()([conv, lstm])
    combined = Dense(64, activation='relu')(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(1, activation='sigmoid')(combined)

    model = Model(inputs=input, outputs=output)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [ ]:
#training of model

model_layer1 = define_model()

# Training parameters
batch_size = 32
epochs = 50

# Train layer 1 model
history1 = model_layer1.fit(
    train_x, 
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,  # Using 20% of training data for validation
    verbose=1
)

# Save the trained model
model_layer1.save_weights(f"/content/Enhancer_BiLSTMAtt-ResNet/Enhancer-LSTMAtt/ResNet+LSTM+Attention({DNA_encoding_scheme}).weights.h5")


In [ ]:
#if not training uncomment next line
# model_layer1 = define_model()
model_layer1.load_weights(f"/content/Enhancer_BiLSTMAtt-ResNet/Enhancer-LSTMAtt/ResNet+LSTM+Attention({DNA_encoding_scheme}).weights.h5")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Your existing model prediction logic
res1 = model_layer1.predict(test_x)  # Prediction
pred1 = np.squeeze(res1, axis=-1)
f1 = pred1 > 0.5
pred1[f1] = 1
pred1[pred1 < 0.6] = 0

# Call to your sn_sp_acc_mcc function (not shown in the code snippet you provided)
sn, sp, acc, mcc, kappa, csi, gmean = sn_sp_acc_mcc(test_y, pred1, pos_label=1)

print(f"Sensitivity (SN): {sn:.4f}")
print(f"Specificity (SP): {sp:.4f}")
print(f"Accuracy (ACC): {acc:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print(f"Critical Success Index (CSI): {csi:.4f}")
print(f"Geometric Mean (G-mean): {gmean:.4f}")

# ROC curve calculations and AUC
FPR_1, TPR_1, threshold_1 = roc_curve(test_y, model_layer1.predict(test_x), pos_label=1)
AUC_1 = auc(FPR_1, TPR_1)
print(AUC_1)

# Plot ROC curve
plt.figure(figsize=(5, 5))
plt.title('ROC curves')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.plot(FPR_1, TPR_1, color='r', label='Enhancer (AUC={:.4f})'.format(AUC_1))
plt.plot([0, 1], [0, 1], color='m', linestyle='--')
plt.legend(loc='lower right')
plt.show()

# --- Confusion Matrix Plotting ---
# Calculate confusion matrix
cm = confusion_matrix(test_y, pred1)

# Create the plot
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', cbar=False,
            xticklabels=['No enhancer', 'Enhancer'], yticklabels=['No enhancer', 'Enhancer'])

# Add labels and title
plt.title('Confusion Matrix for Numeric')
plt.xlabel('Predictions')
plt.ylabel('Actual Values')

# Display the plot
plt.show()


In [ ]:
## Coding for K fold Validation

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPool1D, Bidirectional, LSTM, Dense, Dropout, Concatenate, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam

def define_model():
    maxlen = X.shape[1]  
    max_features = 100  
    embedding_dims = 128
    input = Input(shape=(maxlen,))
    embedding = Embedding(max_features, embedding_dims, input_length=maxlen)(input)

    # Convolutional branch
    conv = Conv1D(64, kernel_size=5, activation='relu')(embedding)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = GlobalMaxPool1D()(conv)

    # LSTM branch
    lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding)
    lstm = GlobalMaxPool1D()(lstm)

    # Combine branches
    combined = Concatenate()([conv, lstm])
    combined = Dense(64, activation='relu')(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(1, activation='sigmoid')(combined)

    model = Model(inputs=input, outputs=output)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

def plot_kfold_roc_curves(X, y, k=5, epochs=50, batch_size=32):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    results = {
        'SN': [],
        'SP': [],
        'ACC': [],
        'MCC': [],
        'AUC': []
    }

    plt.figure(figsize=(8, 6))

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = define_model()
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                  validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

        y_pred = model.predict(X_val).ravel()

        fpr, tpr, _ = roc_curve(y_val, y_pred)
        roc_auc = auc(fpr, tpr)

        sens, spec, acc, mcc = calculate_metrics(y_val, y_pred)

        results['SN'].append(sens)
        results['SP'].append(spec)
        results['ACC'].append(acc)
        results['MCC'].append(mcc)
        results['AUC'].append(roc_auc)

        plt.plot(fpr, tpr, label=f'Fold {fold + 1} (AUC={roc_auc:.4f})', alpha=0.8)

    df = pd.DataFrame(results)
    df.index = [f'Fold {i+1}' for i in range(k)]
    df.loc['Mean'] = df.mean()
    df = df.round(4)

    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.title('ROC Curves')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    print("\nPerformances of 5-fold cross-validation:")
    print("=" * 80)
    print(df.to_string())
    print("=" * 80)

    return df

# Use the function
results_df = plot_kfold_roc_curves(train_x, train_y, k=5)